In [8]:
pip install faiss-cpu

     --------------------------------------- 15.0/15.0 MB 16.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Lan Dao\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [6]:
pip install pymupdf


     --------------------------------------- 16.6/16.6 MB 15.6 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\Lan Dao\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
pip install langchain 

You should consider upgrading via the 'c:\Users\Lan Dao\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
pip install -U langchain-community

     ---------------------------------------- 2.5/2.5 MB 9.5 MB/s eta 0:00:00
     -------------------------------------- 443.2/443.2 KB 9.2 MB/s eta 0:00:00
     ---------------------------------------- 44.4/44.4 KB ? eta 0:00:00
     -------------------------------------- 121.0/121.0 KB 6.9 MB/s eta 0:00:00
     ---------------------------------------- 63.8/63.8 KB ? eta 0:00:00
     ---------------------------------------- 93.5/93.5 KB 5.5 MB/s eta 0:00:00
     ---------------------------------------- 45.7/45.7 KB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 50.9/50.9 KB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\Lan Dao\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [9]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# Step 1: Load PDF
loader = PyMuPDFLoader("data\Huy_Bui_Resume.pdf")  # Replace with your PDF path
documents = loader.load()
print("Number of words:", len(documents[0].page_content.split(" ")))

# Step 2: Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=50)
chunks = splitter.split_documents(documents)

# Step 3: Use HuggingFace Embeddings (open source)
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False} #False Euclidean, True cosine similarity
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Step 4: Create FAISS vector index
vector_store = FAISS.from_documents(chunks, hf)

# Step 5: Save the index locally
vector_store.save_local("faiss_index_open")
print("✅ Indexing complete and saved to 'faiss_index_open'")


Number of words: 360
✅ Indexing complete and saved to 'faiss_index_open'


In [10]:
# Extract raw texts
texts = [chunk.page_content for chunk in chunks[:3]]  # First 3 chunks

# Get embeddings
vectors = hf.embed_documents(texts)

for i, (text, vec) in enumerate(zip(texts, vectors)):
    print(f"\n🔢 Embedding {i+1}:")
    print(f"Text: {text[:100]}...")
    print(f"Vector (first 5 dims): {vec[:5]}")
    print(f"Vector length: {len(vec)}")



🔢 Embedding 1:
Text: Huy Bui
williamhuybui@gmail.com | linkedin.com/in/huy-bui-ds
Experience
Publicis Groupe
Remote
Senio...
Vector (first 5 dims): [-0.01658700592815876, 0.07142695784568787, -0.036695148795843124, -0.064139723777771, -0.004133868962526321]
Vector length: 768


In [11]:
results = vector_store.similarity_search_with_score(
    "Mentor ?", k=1
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=1.609921] Huy Bui
williamhuybui@gmail.com | linkedin.com/in/huy-bui-ds
Experience
Publicis Groupe
Remote
Senior Data Scientist
June 2022 – Present
– Developed 7 client-agnostic applications using Python and Dash, empowering 100+ analysts to streamline data
analysis and reporting, saving over 4,600 hours of productivity.
– Led data science initiatives and cross-functional teams to build scalable analytics solutions, driving alignment and
efficiency in a fast-paced, startup-like environment.
– Built an automated data pipeline leveraging Python, SQL, BigQuery, AWS Lambda, S3, and EventBridge.
– Developed regression models and LLM-based procedures to extract insights and optimize marketing spend
allocation for global enterprises.
– Enhanced UI/UX by migrating Python codebases to React, using tools and techniques such as Figma, Postman,
and REST APIs.
Enovate Upstream
Remote
Research Data Scientist
April 2020 – June 2022
– Lead author of the research Machine Learning Applications to 